In [1]:
import os
import glob
import imageio
import pandas as pd
import geopandas as gpd
import numpy as np
import contextily as cx
import proj as prj
import rasterio as rio
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import xyzservices.providers as xyz
pd.set_option('display.max_columns', None)

### Zipcode Yearly Housing Price

In [2]:
#reading zipcode data
zipcode_FL = gpd.read_file('../data/Floodzones/zipcode_floodzoneratio.shp')
zipcode_FL.head()

,ZCTA5CE20,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,Shape_Leng,Shape_Area,1in100,1in500,geometry
0,34210,34210,B5,G6350,S,23017418.0,26292447.0,+27.4375477,-082.6301776,0.392643,0.004501,0.113379,0.222640,"POLYGON ((-82.60410 27.46737, -82.60410 27.467..."
1,33010,33010,B5,G6350,S,11192368.0,164233.0,+25.8332017,-080.2787199,0.199796,0.001022,0.258887,0.508645,"POLYGON ((-80.27856 25.85130, -80.27659 25.851..."
2,33919,33919,B5,G6350,S,22658042.0,6915450.0,+26.5558907,-081.9028554,0.261206,0.002679,0.150244,0.245144,"POLYGON ((-81.89077 26.59651, -81.89080 26.596..."
3,33461,33461,B5,G6350,S,18418467.0,1128555.0,+26.6185821,-080.0932157,0.240485,0.001772,0.243344,0.415006,"POLYGON ((-80.07217 26.63395, -80.06876 26.633..."
4,33605,33605,B5,G6350,S,20307052.0,3682174.0,+27.9510524,-082.4296347,0.291570,0.002193,0.134586,0.202821,"POLYGON ((-82.43571 27.90303, -82.43620 27.903..."


### Zillow Home Value Index

In [3]:
#reading and slicing florida for Zillow Home Value Index csv data
ZHVI = pd.read_csv('../data/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
ZHVI = ZHVI[ZHVI['State'] == "FL"]
print (ZHVI.RegionID.nunique())
display (ZHVI.head())

927


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,2000-10-31,2000-11-30,2000-12-31,2001-01-31,2001-02-28,2001-03-31,2001-04-30,2001-05-31,2001-06-30,2001-07-31,2001-08-31,2001-09-30,2001-10-31,2001-11-30,2001-12-31,2002-01-31,2002-02-28,2002-03-31,2002-04-30,2002-05-31,2002-06-30,2002-07-31,2002-08-31,2002-09-30,2002-10-31,2002-11-30,2002-12-31,2003-01-31,2003-02-28,2003-03-31,2003-04-30,2003-05-31,2003-06-30,2003-07-31,2003-08-31,2003-09-30,2003-10-31,2003-11-30,2003-12-31,2004-01-31,2004-02-29,2004-03-31,2004-04-30,2004-05-31,2004-06-30,2004-07-31,2004-08-31,2004-09-30,2004-10-31,2004-11-30,2004-12-31,2005-01-31,2005-02-28,2005-03-31,2005-04-30,2005-05-31,2005-06-30,2005-07-31,2005-08-31,2005-09-30,2005-10-31,2005-11-30,2005-12-31,2006-01-31,2006-02-28,2006-03-31,2006-04-30,2006-05-31,2006-06-30,2006-07-31,2006-08-31,2006-09-30,2006-10-31,2006-11-30,2006-12-31,2007-01-31,2007-02-28,2007-03-31,2007-04-30,2007-05-31,2007-06-30,2007-07-31,2007-08-31,2007-09-30,2007-10-31,2007-11-30,2007-12-31,2008-01-31,2008-02-29,2008-03-31,2008-04-30,2008-05-31,2008-06-30,2008-07-31,2008-08-31,2008-09-30,2008-10-31,2008-11-30,2008-12-31,2009-01-31,2009-02-28,2009-03-31,2009-04-30,2009-05-31,2009-06-30,2009-07-31,2009-08-31,2009-09-30,2009-10-31,2009-11-30,2009-12-31,2010-01-31,2010-02-28,2010-03-31,2010-04-30,2010-05-31,2010-06-30,2010-07-31,2010-08-31,2010-09-30,2010-10-31,2010-11-30,2010-12-31,2011-01-31,2011-02-28,2011-03-31,2011-04-30,2011-05-31,2011-06-30,2011-07-31,2011-08-31,2011-09-30,2011-10-31,2011-11-30,2011-12-31,2012-01-31,2012-02-29,2012-03-31,2012-04-30,2012-05-31,2012-06-30,2012-07-31,2012-08-31,2012-09-30,2012-10-31,2012-11-30,2012-12-31,2013-01-31,2013-02-28,2013-03-31,2013-04-30,2013-05-31,2013-06-30,2013-07-31,2013-08-31,2013-09-30,2013-10-31,2013-11-30,2013-12-31,2014-01-31,2014-02-28,2014-03-31,2014-04-30,2014-05-31,2014-06-30,2014-07-31,2014-08-31,2014-09-30,2014-10-31,2014-11-30,2014-12-31,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28
80,73221,80,34787,zip,FL,FL,Winter Garden,"Orlando-Kissimmee-Sanford, FL",Orange County,172048.805635,173031.292114,173844.579523,175281.051126,176267.979252,176779.429058,177198.518172,177812.560501,179117.109299,180678.584902,182037.737672,183061.558299,183814.685665,184323.005012,184791.416840,185475.019599,186252.590426,186933.763896,187283.118270,188035.630653,189224.159684,190674.243943,191885.031289,193029.099674,194329.178201,195692.434931,196990.682806,198111.341461,199428.305918,200626.140417,201707.513475,202957.020495,204104.849231,205537.859950,206938.203357,208256.312219,209255.769590,210037.593817,210898.944816,211958.380719,212757.021738,213809.730887,214881.900420,216431.783908,218146.646775,220093.443186,222072.640826,223904.244024,225807.732244,227848.383004,229823.304830,232221.419487,234782.355625,237829.690706,240844.367816,24467

In [4]:
#changing Zipcode information datatype to join later
zipcode_FL['ZCTA5CE20'] = zipcode_FL['ZCTA5CE20'].astype(int)

#### Taking yearly average of home index values

In [5]:
#removing month and date from columns
for col in ZHVI.iloc[:, 9:].columns:
    ZHVI.rename(columns={col:col[0:4]},inplace=True)
print (ZHVI.columns)

Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'City', 'Metro', 'CountyName', '2000',
       ...
       '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2023',
       '2023'],
      dtype='object', length=287)


In [6]:
# yearly mean calculation for year columns only
year_only =  ZHVI.iloc[:,9:]
yearly_mean = year_only.groupby(level=0,axis=1).mean()


In [7]:
# concating yearly mean with rest of zhvi columns
zhvi = pd.concat([(ZHVI.iloc[:, 0:8]),yearly_mean], axis =1)
#display (zhvi.head())                   

In [8]:
#merging zhvi with zip code boundaries of florida
df1_zhvi = zhvi.merge(zipcode_FL, left_on = 'RegionName', right_on = 'ZCTA5CE20')
print (df1_zhvi.RegionName.nunique())
df1_zhvi = gpd.GeoDataFrame(df1_zhvi, geometry = 'geometry')
df1_zhvi.head()

927


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,ZCTA5CE20,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,Shape_Leng,Shape_Area,1in100,1in500,geometry
0,73221,80,34787,zip,FL,FL,Winter Garden,"Orlando-Kissimmee-Sanford, FL",177263.267129,187643.480413,201223.320205,215354.008392,240804.785914,301062.446501,383426.864196,379847.503394,312856.581220,237040.293736,207987.791511,192163.891302,194907.515570,225620.583412,265467.843769,284071.020341,301703.295696,317279.663610,336767.663471,362629.331745,384242.676969,441157.217430,546216.226622,553544.350791,34787,34787,B5,G6350,S,174167301.0,27808565.0,+28.4851744,-081.6194145,1.145243,0.018612,0.174195,0.219499,"MULTIPOLYGON (((-81.54431 28.52978, -81.54594 ..."
1,72362,96,33024,zip,FL,FL,Pembroke Pines,"Miami-Fort Lauderdale-Pompano Beach, FL",95537.304993,108702.206559,124185.919386,139790.268667,163857.909835,205683.131541,251764.265875,253794.935348,200817.126138,143575.945011,131565.666263,121491.664476,126801.369978,149960.265712,175041.638674,197509.653576,221028.288873,249445.718620,274301.429368,290352.267248,308487.613036,347410.141244,407862.981147,424388.837688,33024,33024,B5,G6350,S,30052050.0,738934.0,+26.0263576,-080.2446326,0.249723,0.002777,0.234319,0.422797,"POLYGON ((-80.21425 26.04301, -80.21424 26.042..."
2,72530,133,33311,zip,FL,FL,Fort Lauderdale,"Miami-Fort Lauderdale-Pompano Beach, FL",54515.456786,62814.582286,70907.582963,77471.104988,89831.197604,112318.043285,136781.857835,145739.291901,119516.244525,83129.706554,71682.686912,60706.506165,58555.923525,69627.607064,87321.086095,102459.049640,119528.897512,145298.003483,171881.436508,193133.086193,212828.248717,247273.325587,299024.079330,315607.724772,33311,33311,B5,G6350,S,26860151.0,449109.0,+26.1442083,-080.1727850,0.251871,0.002465,0.258111,0.433085,"POLYGON ((-80.14644 26.16659, -80.14644 26.166..."
3,72361,145,33023,zip,FL,FL,Miramar,"Miami-Fort Lauderdale-Pompano Beach, FL",80083.825376,90701.811936,103442.366702,115868.261262,135242.110348,167436.299937,204958.318773,215020.922953,176653.261455,126250.664213,111318.280962,101894.465529,101900.647392,120525.107732,143090.469411,162643.985334,187322.114906,219517.845182,249539.852448,268765.281952,289112.685012,327509.938242,382882.877666,398303.546514,33023,33023,B5,G6350,S,22801303.0,416222.0,+25.9884843,-080.2124621,0.239457,0.002093,0.234619,0.422255,"POLYGON ((-80.17512 25.99206, -80.17507 25.990..."
4,72578,149,33411,zip,FL,FL,NaN,"Miami-Fort Lauderdale-Pompano Beach, FL",121188.331699,131454.610877,147881.735614,170471.071814,203954.143763,257885.132880,307094.337425,284740.843805,228694.481712,183239.388321,166952.262721,152492.785203,148503.083351,170190.526392,203451.330663,226601.429539,252266.056058,272725.111759,291440.045873,303474.964079,317310.946182,365624.760036,440075.865111,447611.669489,33411,33411,B5,G6350,S,90267008.0,3511121.0,+26.7152013,-080.2037974,0.545057,0.008506,0.406732,0.582441,"POLYGON ((-80.13427 26.75174, -80.13430 26.750..."


#### Making yearly gif map

In [9]:
# save all the maps in the charts folder
output_path = '../data/pngmaps'

# counter for the for loop
i = 0

# list of years (which are the column names at the moment)
list_of_years = ['2000','2001','2002','2003', '2004', '2005',
                '2006', '2007', '2008', '2009', '2010', '2011',
                '2012', '2013', '2014', '2015', '2016', '2017',
                '2018', '2019', '2020', '2021', '2022', '2023']

In [10]:
# set the min and max range for the choropleth map
vmin, vmax = 2000, 2023

In [11]:
# start the for loop to create one map per year
for year in list_of_years:
    
    # create map, UDPATE: added plt.Normalize to keep the legend range the same for all maps
    fig = zipcode_FL.plot( color='k', linewidth=1, figsize=(9,9), alpha = 0.05, edgecolor='k')
    df1_zhvi.plot(ax = fig, column=year,cmap='viridis', 
                        figsize=(10,10), scheme = 'quantiles', linewidth=0.5,legend = True, legend_kwds = {'loc': 'lower left'})
    #cx.add_basemap(fig, source=xyz.OpenStreetMap.Mapnik, crs=zipcode_FL.crs)
 

    # remove axis of chart
    fig.axis('off')
    
    # add a title
    fig.set_title(year +' Average Housing Price for Zipcodes',
              fontdict={'fontsize': '20',
                         'fontweight' : '3'})
    
    # this will save the figure as a high-res png in the output path. you can also save as svg if you prefer.
    filepath = os.path.join(output_path, year +'Zipcodeprice.png')
    chart = fig.get_figure()
    chart.savefig(filepath, dpi=300)
    
     # position the annotation to the bottom left
    fig.annotate(year,
            xy=(0.1, .225), xycoords='figure fraction',
            horizontalalignment='left', verticalalignment='top',
            fontsize=35)
    plt.close()

In [12]:
# Create Gif file

image_path = r"C:\Users\bhandarir\Desktop\USF_GraduateSchool-Radhika\Zillow Housing\data\pngmaps\*.png"

images = glob.glob(image_path)
#print (images)
#list(image_path.glob('*.png'))
image_list = []
for file_name in images:
    image_list.append(imageio.v2.imread(file_name))
    #os.remove(file_name)
    
imageio.mimwrite('../data/pngmaps/Zipcode_GifMap.gif', image_list, duration = 500)

### Neighborhood Yearly Housing Price

In [13]:
#reading neighborhood data,with floodzone ratio
neighborhood_FL = gpd.read_file('../data/Floodzones/neighborhood_floodzoneratio.shp')
neighborhood_FL.head()
#has only 371 counties

,State,County,City,Name,RegionID,Shape_Leng,Shape_Area,1in100,1in500,geometry
0,FL,Orange,Orlando,Alafaya,15046,0.551708,0.009048,0.219424,0.285221,"POLYGON ((-81.11414 28.47162, -81.11413 28.471..."
1,FL,Broward,Hollywood,Hillcrest,25125,0.055430,0.000146,0.240506,0.363973,"POLYGON ((-80.19527 25.99527, -80.19519 25.997..."
2,FL,Miami-Dade,Miami,Brickell,55477,0.086324,0.000260,0.112576,0.175456,"POLYGON ((-80.19729 25.76873, -80.19681 25.768..."
3,FL,Pasco,New Port Richey,North Park,116942,0.039007,0.000065,0.083825,0.087470,"POLYGON ((-82.70738 28.25927, -82.70726 28.259..."
4,FL,Volusia,New Smyrna Beach,South West,121790,0.100864,0.000253,0.281463,0.471792,"POLYGON ((-80.93372 29.01789, -80.93372 29.018..."


In [14]:
#changing data type of RegionID to merge later
neighborhood_FL['RegionID'] = neighborhood_FL['RegionID'].astype(int)
print (neighborhood_FL.County.nunique())

27


In [15]:
#reading Zillow Home Values Index for Neighborhood
ZHVI_neighborhood = pd.read_csv('../data/Neighborhood_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
#filtering florida
ZHVI_neighborhood = ZHVI_neighborhood[ZHVI_neighborhood['State'] == "FL"]
ZHVI_neighborhood['CountyName'] = ZHVI_neighborhood['CountyName'].apply(lambda x: x.rsplit(' ', 1)[0])
ZHVI_neighborhood.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,2000-10-31,2000-11-30,2000-12-31,2001-01-31,2001-02-28,2001-03-31,2001-04-30,2001-05-31,2001-06-30,2001-07-31,2001-08-31,2001-09-30,2001-10-31,2001-11-30,2001-12-31,2002-01-31,2002-02-28,2002-03-31,2002-04-30,2002-05-31,2002-06-30,2002-07-31,2002-08-31,2002-09-30,2002-10-31,2002-11-30,2002-12-31,2003-01-31,2003-02-28,2003-03-31,2003-04-30,2003-05-31,2003-06-30,2003-07-31,2003-08-31,2003-09-30,2003-10-31,2003-11-30,2003-12-31,2004-01-31,2004-02-29,2004-03-31,2004-04-30,2004-05-31,2004-06-30,2004-07-31,2004-08-31,2004-09-30,2004-10-31,2004-11-30,2004-12-31,2005-01-31,2005-02-28,2005-03-31,2005-04-30,2005-05-31,2005-06-30,2005-07-31,2005-08-31,2005-09-30,2005-10-31,2005-11-30,2005-12-31,2006-01-31,2006-02-28,2006-03-31,2006-04-30,2006-05-31,2006-06-30,2006-07-31,2006-08-31,2006-09-30,2006-10-31,2006-11-30,2006-12-31,2007-01-31,2007-02-28,2007-03-31,2007-04-30,2007-05-31,2007-06-30,2007-07-31,2007-08-31,2007-09-30,2007-10-31,2007-11-30,2007-12-31,2008-01-31,2008-02-29,2008-03-31,2008-04-30,2008-05-31,2008-06-30,2008-07-31,2008-08-31,2008-09-30,2008-10-31,2008-11-30,2008-12-31,2009-01-31,2009-02-28,2009-03-31,2009-04-30,2009-05-31,2009-06-30,2009-07-31,2009-08-31,2009-09-30,2009-10-31,2009-11-30,2009-12-31,2010-01-31,2010-02-28,2010-03-31,2010-04-30,2010-05-31,2010-06-30,2010-07-31,2010-08-31,2010-09-30,2010-10-31,2010-11-30,2010-12-31,2011-01-31,2011-02-28,2011-03-31,2011-04-30,2011-05-31,2011-06-30,2011-07-31,2011-08-31,2011-09-30,2011-10-31,2011-11-30,2011-12-31,2012-01-31,2012-02-29,2012-03-31,2012-04-30,2012-05-31,2012-06-30,2012-07-31,2012-08-31,2012-09-30,2012-10-31,2012-11-30,2012-12-31,2013-01-31,2013-02-28,2013-03-31,2013-04-30,2013-05-31,2013-06-30,2013-07-31,2013-08-31,2013-09-30,2013-10-31,2013-11-30,2013-12-31,2014-01-31,2014-02-28,2014-03-31,2014-04-30,2014-05-31,2014-06-30,2014-07-31,2014-08-31,2014-09-30,2014-10-31,2014-11-30,2014-12-31,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31
52,15046,52,Alafaya,neighborhood,FL,FL,Orlando,"Orlando-Kissimmee-Sanford, FL",Orange,171436.243312,171816.273344,172451.491027,173497.255161,174711.441618,175852.457123,177174.438049,178612.802052,180218.941566,181899.105333,183551.393090,185131.854498,186506.057585,187645.300561,188675.430150,189848.356299,191320.316663,192825.478172,194181.228578,195410.279374,196510.223483,197725.131243,198853.093920,199992.905546,200902.303598,201745.804522,202549.525228,203369.747625,204264.239000,205381.728218,206673.613535,208209.615528,209638.954110,211134.531482,212507.363836,213869.709368,215130.395592,216261.947613,217547.267471,219045.500068,220632.861851,222158.130463,223601.757562,224987.212630,226092.407437,227167.979956,228496.414618,230066.476073,231589.825861,233063.356902,234632.857420,236630.985632,238890.395642,241486.346815,244297.615

#### Taking yearly average of home index values

In [16]:
#removing month and date from columns
for col in ZHVI_neighborhood.iloc[:, 9:].columns:
    ZHVI_neighborhood.rename(columns={col:col[0:4]},inplace=True)
print (ZHVI_neighborhood.columns)

Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'City', 'Metro', 'CountyName', '2000',
       ...
       '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2023', '2023',
       '2023'],
      dtype='object', length=288)


In [17]:
# yearly mean calculation for year columns only
year_only =  ZHVI_neighborhood.iloc[:,9:]
yearly_mean = year_only.groupby(level=0,axis=1).mean()

In [18]:
# concating yearly mean with rest of zhvi columns
zhvi_n = pd.concat([(ZHVI_neighborhood.iloc[:, 0:8]),yearly_mean], axis =1)
display (zhvi_n.head())                   

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
52,15046,52,Alafaya,neighborhood,FL,FL,Orlando,"Orlando-Kissimmee-Sanford, FL",177196.141348,193291.150131,206687.261337,222599.029278,244726.810935,305999.456645,386548.387657,370623.515716,298109.011991,225637.025668,204715.032999,187845.366291,191702.394224,221287.234495,251894.148158,265014.376969,281208.330584,299661.294574,319900.350122,332666.734710,347846.351456,385652.530232,447293.885714,449117.123993
72,25372,72,Kendall,neighborhood,FL,FL,Miami,"Miami-Fort Lauderdale-Pompano Beach, FL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,291682.624114,306548.735528,330571.390847,337365.367324,348564.987310,389364.548306,457624.630843,474528.578410
81,31789,81,Golden Gate,neighborhood,FL,FL,Naples,"Naples-Marco Island, FL",137181.615799,153671.126402,172954.637867,195598.218823,225137.397765,302992.680581,382182.668014,369537.838653,290824.099888,185748.202121,161967.328259,150246.964998,149357.322488,173789.986422,205171.836424,241959.361499,277786.976142,292463.119794,306020.523782,320069.296460,334546.936395,396415.891871,494121.977552,500775.269724
148,124381,150,Flagami,neighborhood,FL,FL,Miami,"Miami-Fort Lauderdale-Pompano Beach, FL",84395.864198,92381.703497,104735.135784,122522.909376,144105.477357,181785.679435,224151.315861,237570.742483,199779.507208,137283.514352,116522.511410,103226.841018,103792.859872,123722.912642,145803.786240,169490.495089,195624.628176,219190.958956,242191.702786,256942.374326,276470.453941,311783.520822,373291.450560,400405.204235
166,269072,168,Little Havana,neighborhood,FL,FL,Miami,"Miami-Fort Lauderdale-Pompano Beach, FL",49015.982121,52388.388264,58776.028562,67618.281020,80283.238789,104335.310097,129068.947550,134979.502080,113979.521757,85327.209068,65097.363203,56581.945261,58592.531285,72507.128407,90065.001656,107312.144733,123870.626912,138161.328104,156887.664106,167655.043318,179775.779361,203133.909101,269100.333011,293675.456158


In [19]:
#checking number of null values 
noPrice_2000 = zhvi_n[zhvi_n['2000'].isna()]
print (noPrice_2000.shape[0], " number of neighborhoods have no price for the year 2000")

950  number of neighborhoods have no price for the year 2000


In [20]:
# dropping rows with null values for 2000
zhvi_n.dropna(subset = '2000', inplace = True)

In [21]:
#merging zhvi with zip code boundaries of florida
df1_zhvi_n = zhvi_n.merge(neighborhood_FL[['RegionID', '1in100', '1in500', 'geometry']], on = 'RegionID')
print (df1_zhvi.RegionID.nunique())
df1_zhvi_n = gpd.GeoDataFrame(df1_zhvi_n, geometry = 'geometry')
df1_zhvi_n.head()

927


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,1in100,1in500,geometry
0,15046,52,Alafaya,neighborhood,FL,FL,Orlando,"Orlando-Kissimmee-Sanford, FL",177196.141348,193291.150131,206687.261337,222599.029278,244726.810935,305999.456645,386548.387657,370623.515716,298109.011991,225637.025668,204715.032999,187845.366291,191702.394224,221287.234495,251894.148158,265014.376969,281208.330584,299661.294574,319900.350122,332666.734710,347846.351456,385652.530232,447293.885714,449117.123993,0.219424,0.285221,"POLYGON ((-81.11414 28.47162, -81.11413 28.471..."
1,124381,150,Flagami,neighborhood,FL,FL,Miami,"Miami-Fort Lauderdale-Pompano Beach, FL",84395.864198,92381.703497,104735.135784,122522.909376,144105.477357,181785.679435,224151.315861,237570.742483,199779.507208,137283.514352,116522.511410,103226.841018,103792.859872,123722.912642,145803.786240,169490.495089,195624.628176,219190.958956,242191.702786,256942.374326,276470.453941,311783.520822,373291.450560,400405.204235,0.335167,0.512971,"POLYGON ((-80.24146 25.79374, -80.24018 25.792..."
2,269072,168,Little Havana,neighborhood,FL,FL,Miami,"Miami-Fort Lauderdale-Pompano Beach, FL",49015.982121,52388.388264,58776.028562,67618.281020,80283.238789,104335.310097,129068.947550,134979.502080,113979.521757,85327.209068,65097.363203,56581.945261,58592.531285,72507.128407,90065.001656,107312.144733,123870.626912,138161.328104,156887.664106,167655.043318,179775.779361,203133.909101,269100.333011,293675.456158,0.298584,0.439440,"POLYGON ((-80.21448 25.78252, -80.21375 25.782..."
3,269228,231,Alameda - West Flagler,neighborhood,FL,FL,Miami,"Miami-Fort Lauderdale-Pompano Beach, FL",102392.874664,112323.336503,127048.231204,150048.174259,180458.461724,227672.376890,277030.733588,288276.894115,236637.534695,163894.798699,141918.446684,130336.560438,128452.956624,150226.174002,173889.849311,204933.226302,235032.494823,259800.144912,281418.100736,301226.209714,327273.580949,371103.508260,447147.189527,474287.318351,0.160301,0.296925,"POLYGON ((-80.23116 25.77915, -80.23115 25.778..."
4,123423,238,Allapattah,neighborhood,FL,FL,Miami,"Miami-Fort Lauderdale-Pompano Beach, FL",56158.149368,62262.818677,68510.284290,82662.728702,97103.589089,122317.260226,153739.795865,165256.279767,141263.405257,98838.174477,70171.583946,53421.773254,51031.439585,61749.441672,77900.840212,95935.836924,114414.283114,138238.955246,164419.168936,187211.758965,214671.062598,250556.723225,308774.526891,338416.409544,0.302776,0.447183,"POLYGON ((-80.20589 25.81227, -80.20597 25.810..."


#### Making yearly gif map

In [22]:
# start the for loop to create one map per year
for year in list_of_years:
    
    # create map, UDPATE: added plt.Normalize to keep the legend range the same for all maps
    fig = neighborhood_FL.plot( color='k', linewidth=1, figsize=(9,9), alpha = 0.05, edgecolor='k')
    df1_zhvi_n.plot(ax = fig, column=year,cmap='viridis', 
                        figsize=(10,10), scheme = 'quantiles', linewidth=0.5,legend = True, legend_kwds = {'loc': 'lower left'})
    #cx.add_basemap(fig, source=xyz.OpenStreetMap.Mapnik, crs=neighborhood_FL.crs)
 

    # remove axis of chart
    fig.axis('off')
    
    # add a title
    fig.set_title(year +' Average Housing Price for Neighborhoods',
              fontdict={'fontsize': '20',
                         'fontweight' : '3'})
    
    # this will save the figure as a high-res png in the output path. you can also save as svg if you prefer.
    filepath = os.path.join(output_path, year +'Neighborhoodprice.png')
    chart = fig.get_figure()
    chart.savefig(filepath, dpi=300)
    
     # position the annotation to the bottom left
    fig.annotate(year,
            xy=(0.1, .225), xycoords='figure fraction',
            horizontalalignment='left', verticalalignment='top',
            fontsize=35)
    plt.close()

In [23]:
# Create Gif file

image_path = r"C:\Users\bhandarir\Desktop\USF_GraduateSchool-Radhika\Zillow Housing\data\pngmaps\*Neighborhoodprice.png"

images = glob.glob(image_path)
#print (images)
#list(image_path.glob('*.png'))
image_list = []
for file_name in images:
    image_list.append(imageio.v2.imread(file_name))
    #os.remove(file_name)
    
imageio.mimwrite('../data/pngmaps/Neighborhood_GifMap.gif', image_list, duration = 500)